### **ETL process (Extract, Transform & Load) - Plataforma de videojuegos Stream** 
#### **DATOS: User_Items**

Empezaremos por entender, transfomar y disponibilizar, encontrando problemas en los datos, aplicando tecnicas de limpieza y preprocesamiente. Finalmente, almacenaremos los datos transformatos para futuras exploraciones.  

Entenderemos el contexto y la informacion necesaria para nuestro analisis, tratando consistencia a esos datos.

#### **Importamos las librerias necesarias**

In [6]:
import numpy as np # Para trabajar con arrays y matrices


import pandas as pd #para el analisis de tablas tabulares
from pandas import json_normalize 

import os # OS module help us interact with the operative sistem

# Gdown makes easy to download files from Google Drive with its ID 
import gdown

# Json its a commonly used format for exchange of data
import json


import warnings
warnings.filterwarnings("ignore")

import ast

### 2º manera de subir el raw: ESTO es para subir desde un gzip raw

In [7]:
# import gzip 
# import ast
# import pandas as pd
# data = []
# try:
#     with gzip.open('users_items.json.gz', 'rb') as d:
#         for line in d:
#             data.append (ast.literal_eval(line.decode('utf-8')))
#     Users_Items = pd.DataFrame (data)
# finally:
#     d.close()

## **1. Cargamos el Dataset**
Fuente de datos: **australian_user_items.json**

Vamos a descargar el archivo de forma local, ya que es necesario cargarlo de una manera especifica, debido a su estructura. 

Este archivo está disponible en Google Drive con acceso compartido. Puede descargarlo en el siguiente enlace:

Datasets: https://drive.google.com/uc?export=download&id=1BzbeJv1878pa2lQMspr5y03dUdIXaYyX


In [8]:
ruta = 'users_items.json'

In [9]:
with open(ruta, 'r', encoding='utf-8') as f:
    data = f.readlines()

# vamos a convertir las lineas en registros
records = [eval(line.strip()) for line in data]

# Creamos el DataFrame a partir de los records o registros
df_UserItems= pd.DataFrame(records)

In [10]:
respaldo = df_UserItems

## **2. Explorar y entender el conjunto de datos**

In [11]:
# traemos una vista rapida de la tabla

print('DataFrame - First & Last Rows:')
df_UserItems

DataFrame - First & Last Rows:


,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."
...,...,...,...,...,...
88305,76561198323066619,22,76561198323066619,http://steamcommunity.com/profiles/76561198323...,"[{'item_id': '413850', 'item_name': 'CS:GO Pla..."
88306,76561198326700687,177,76561198326700687,http://steamcommunity.com/profiles/76561198326...,"[{'item_id': '11020', 'item_name': 'TrackMania..."
88307,XxLaughingJackClown77xX,0,76561198328759259,http://steamcommunity.com/id/XxLaughingJackClo...,[]
88308,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...,"[{'item_id': '304930', 'item_name': 'Unturned'..."


### **# Analisis Primario**
1. El dataset 'User_items' no muestra datos nulos en ninguna columna.

2. El proceso de recoleccion de datos esta estructurado de una manera optimizada ya que no hay ningun valor faltante. 

3. La columna 'items' tiene una lista anidada

4. Se realizara hacer un analisis mas profundo. 

In [12]:
# Vamos a conocer cuantas Filas tienen nulos en todas sus columnas

print(f'total filas de games_df: {df_UserItems.shape[0]}')
print(f'total filas de games_df con nulos en todas sus columnas: {df_UserItems.isnull().all(axis=1).sum()}')


total filas de games_df: 88310
total filas de games_df con nulos en todas sus columnas: 0


In [13]:
# Obtener información general del DataFrame
print("Información general del DataFrame:\n")
df_UserItems.info()

Información general del DataFrame:

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


### **# Insights**
1. La columna **'items'** contiene una lista con datos anidados (lista de diccionarios).

2. Se va a normalizar la columna. Con esto vamos a obtener una columna por cada clave en ese diccionario, mandentremos la trazabilidad de las columnas **('steam_id', 'items_count', 'user_id' y 'user_url')**
   



In [14]:
# Normalizar la columna 'items'
df_UserItems = pd.json_normalize(records, record_path=['items'], meta=['steam_id','items_count','user_id','user_url'] )

In [15]:
df_UserItems

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [16]:
# buscamos los registros duplicados
duplicados = df_UserItems[df_UserItems.duplicated(subset=['steam_id','item_id','playtime_forever'])]
duplicados.shape

(59117, 8)

In [17]:
#Borramos dulplicados
df_UserItems = df_UserItems.drop_duplicates(keep='first')

In [18]:
# Convertir la columna 'playtime_forever' de minutos a horas
df_UserItems['hours_game'] = df_UserItems['playtime_forever'] / 60

# Redondear los valores a dos decimales (opcional)
df_UserItems['hours_game'] = df_UserItems['hours_game'].round(2)

In [19]:
# Obtener los 5 valores máximos
top_5_max = df_UserItems.nlargest(5, 'hours_game')

# Obtener los 5 valores mínimos
top_5_min = df_UserItems.nsmallest(5, 'hours_game')

# Mostrar los resultados
print("Top 5 valores máximos:")
print(top_5_max[['hours_game']])

print("\nTop 5 valores mínimos:")
print(top_5_min[['hours_game']])

Top 5 valores máximos:
         hours_game
587715     10712.88
2499068    10588.25
4075729    10540.87
1495340    10223.52
1836985    10001.13

Top 5 valores mínimos:
   hours_game
1         0.0
3         0.0
4         0.0
5         0.0
6         0.0


In [20]:
# Contar las filas con valor 0 en la columna 'hours_game'
filas_con_cero = (df_UserItems['hours_game'] == 0).sum()

# Imprimir el resultado
print(f"Existen {filas_con_cero} filas con valor 0 en la columna 'hours_game'.")

Existen 1847730 filas con valor 0 en la columna 'hours_game'.


In [21]:
# Eliminar las filas con valor 0 en la columna 'hours_game'
df_UserItems = df_UserItems[df_UserItems['hours_game'] != 0]

#Eliminamos columna no relevantes
df_UserItems = df_UserItems.drop(['playtime_2weeks','user_url','items_count','item_name','steam_id','playtime_forever'], axis=1)
df_UserItems.columns

Index(['item_id', 'user_id', 'hours_game'], dtype='object')

### **# Insights**
1. Se eliminan los registros con valor nulo en esta columna por motivos de optimizacion de consulta.
   
2. Se encuentran 1'847,730 registros con un valor de 0 en la columna **'hours_game'**.

3. Hay columas irrelevantes, que tambien se remueven por motivos de optimizacion de consultas. 
   



In [22]:
df_UserItems

,item_id,user_id,hours_game
0,10,76561197970982479,0.10
2,30,76561197970982479,0.12
8,300,76561197970982479,78.88
9,240,76561197970982479,30.88
10,3830,76561197970982479,5.55
...,...,...,...
5153202,304930,76561198329548331,11.28
5153203,227940,76561198329548331,0.72
5153206,388490,76561198329548331,0.05
5153207,521570,76561198329548331,0.07


In [23]:
df_UserItems.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3246375 entries, 0 to 5153208
Data columns (total 3 columns):
 #   Column      Dtype  
---  ------      -----  
 0   item_id     object 
 1   user_id     object 
 2   hours_game  float64
dtypes: float64(1), object(2)
memory usage: 99.1+ MB


### 4. Guardamos los datos limpios
Guardamos estos archivos en los siguientes formatos para disponibilizarlos. CSV/ Json/ Parquet

In [25]:
# Los archivos se almacenan en local 
# df_UserItems.to_csv('user_items_limpios.csv', index=False)
# df_UserItems.to_json('user_items_limpios.json', orient='records', lines=True)
df_UserItems.to_parquet('../Data_Files/DF_limpios/2_user_items_limpios.parquet', index=False)